In [1]:
pip install fastapi uvicorn httpx
# Install FastAPI framework for building APIs
# Install Uvicorn ASGI server to run the FastAPI app
# Install HTTPX for making asynchronous HTTP requests




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00


In [4]:
# Import necessary modules from FastAPI and standard libraries
from fastapi import FastAPI, Query
import httpx  # For making asynchronous HTTP requests
from typing import List, Dict  # For type hinting

# Create FastAPI app instance
app = FastAPI()

# Base URL for all test server API requests
BASE_URL = "http://20.244.56.144/evaluation-service"

# Route to get top 5 users with the highest number of posts
@app.get("/users")
async def get_top_users():
    # Asynchronously fetch users and posts from the test server
    async with httpx.AsyncClient() as client:        # Asynchronously fetch users and posts from the test server
        users_response = await client.get(f"{BASE_URL}/users")    # Asynchronously fetch users and posts from the test server
        posts_response = await client.get(f"{BASE_URL}/posts")          # Asynchronously fetch users and posts from the test server

    # Check if both responses are successful
    if users_response.status_code != 200 or posts_response.status_code != 200:
        return {"error": "Failed to fetch users or posts."}

    # Parse the JSON responses into Python dictionaries/lists
    users_data = users_response.json()
    posts_data = posts_response.json()

    # Create a dictionary to count posts per user
    post_counts = {}
    for post in posts_data:
        user_id = post.get("userId")
        post_counts[user_id] = post_counts.get(user_id, 0) + 1  # Increment post count

    # Sort users by number of posts in descending order and get top 5
    top_users = sorted(post_counts.items(), key=lambda x: x[1], reverse=True)[:5]

    # Collect detailed info (id, name, post count) for the top users
    top_user_details = []
    for uid, _ in top_users:
        name = users_data.get(str(uid), "Unknown")  # Get user name from user ID
        top_user_details.append({"id": uid, "name": name, "posts": post_counts[uid]})

    # Return list of top 5 users
    return {"topUsers": top_user_details}


# Route to get either 'popular' or 'latest' posts based on query param
@app.get("/posts")
async def get_posts(type: str = Query(..., regex="^(popular|latest)$")):
    # Asynchronously fetch all posts and comments
    async with httpx.AsyncClient() as client:        # Asynchronously fetch all posts and comments
        posts_response = await client.get(f"{BASE_URL}/posts")   # Asynchronously fetch all posts and comments
        comments_response = await client.get(f"{BASE_URL}/comments")             # Asynchronously fetch all posts and comments

    # Check if both responses are successful
    if posts_response.status_code != 200 or comments_response.status_code != 200:      # Check if both responses are successful
        return {"error": "Failed to fetch posts or comments."}  # Check if both responses are successful

    # Parse JSON data
    posts = posts_response.json()   # Parse JSON data
    comments = comments_response.json()          # Parse JSON data

    # Handle "latest" query type: return 5 most recent posts
    if type == "latest":
        # Sort posts by timestamp descending (latest first)
        sorted_posts = sorted(posts, key=lambda x: x.get("timestamp", ""), reverse=True)    # Sort posts by timestamp descending (latest first)
        return {"latestPosts": sorted_posts[:5]}  # Return top 5 latest posts

    # For "popular" query type: calculate comment count per post
    comment_count = {}         # For "popular" query type: calculate comment count per post
    for c in comments:          # For "popular" query type: calculate comment count per post
        pid = c.get("postId")        # For "popular" query type: calculate comment count per post
        comment_count[pid] = comment_count.get(pid, 0) + 1  # Count comments per post ID

    # Find the highest comment count among all posts
    max_comments = max(comment_count.values(), default=0)

    # Get all posts that have the highest comment count
    popular_posts = [p for p in posts if comment_count.get(p["id"], 0) == max_comments]

    # Return popular posts
    return {"popularPosts": popular_posts}     # Return popular posts


<ipython-input-4-be1983d08440>:49: DeprecationWarning: `regex` has been deprecated, please use `pattern` instead
  async def get_posts(type: str = Query(..., regex="^(popular|latest)$")):
